## About Dataset
### LGG Segmentation Dataset
Dataset used in:

Mateusz Buda, AshirbaniSaha, Maciej A. Mazurowski "Association of genomic subtypes of lower-grade gliomas with shape features automatically extracted by a deep learning algorithm." Computers in Biology and Medicine, 2019.

and

Maciej A. Mazurowski, Kal Clark, Nicholas M. Czarnek, Parisa Shamsesfandabadi, Katherine B. Peters, Ashirbani Saha "Radiogenomics of lower-grade glioma: algorithmically-assessed tumor shape is associated with tumor genomic subtypes and patient outcomes in a multi-institutional study with The Cancer Genome Atlas data." Journal of Neuro-Oncology, 2017.

This dataset contains brain MR images together with manual FLAIR abnormality segmentation masks.
The images were obtained from The Cancer Imaging Archive (TCIA).
They correspond to 110 patients included in The Cancer Genome Atlas (TCGA) lower-grade glioma collection with at least fluid-attenuated inversion recovery (FLAIR) sequence and genomic cluster data available.
Tumor genomic clusters and patient data is provided in `data.csv` file.

## Import packages

In [ ]:
import os
# https://stackoverflow.com/questions/72740907/tensorflow-cant-apply-sharing-policy-file-when-using-mirrored-strategy
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # this MUST come before any tf call.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import cv2
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, Conv2D, Conv2DTranspose, MaxPooling2D, 
                                     Dropout, Activation, BatchNormalization, concatenate)
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.regularizers import l2
import tensorflow_addons as tfa

## EDA

### Get original image and mask image paths

In [ ]:
imgs, masks = [], []
for dirname, _, filenames in os.walk("/kaggle/input/lgg-mri-segmentation/kaggle_3m"):
    for filename in filenames:
        if 'mask'in filename:
            masks.append(os.path.join(dirname, filename))
            imgs.append(os.path.join(dirname, filename.replace('_mask', '')))

print(masks[:3])
print(imgs[:3])

### Create a dataframe

In [ ]:
df = pd.DataFrame({"img_path": imgs, "mask_path": masks})
def positiv_negativ_diagnosis(mask_path):
    value = np.max(cv2.imread(mask_path))
    return 1 if value > 0 else 0

df["diagnosis"] = df["mask_path"].map(positiv_negativ_diagnosis)

In [ ]:
# Check if there is a corresponding image in the same row
print(df.iloc[50].img_path)
print(df.iloc[50].mask_path)
# Number of images with low-grade gliomas
print(df["diagnosis"].value_counts())
df.head()

### Check image shape

In [ ]:
def print_img_shape_randomly_chosen(df, n=10):
    print("\t\t\t   Image Shape\t\t\t  Mask Shape")
    for row in df.sample(n).itertuples():
        print(
            f"Randomly Chosen Index {row.Index}: "
            + f"{get_img_shape(row.img_path)}\t\t"
            + f"{get_img_shape(row.mask_path)}\n"
        )

def get_img_shape(img_path):
    img = cv2.imread(img_path)
    return img.shape

print_img_shape_randomly_chosen(df)

### Visualize images

In [ ]:
def show_image_randomly_chosen(df, n=5):
    fig, axs = plt.subplots(n, 3, figsize=[15, 15])
    axs[0, 0].set_title("Brain MRI Slice")
    axs[0, 1].set_title('Mask')
    axs[0, 2].set_title('Brain MRI Slice with Mask')
    for i, row in enumerate(df.sample(n).itertuples()):
        img = cv2.imread(row.img_path)
        mask = cv2.imread(row.mask_path)
        
        axs[i, 0].imshow(img)
        
        axs[i, 1].imshow(mask)
        
        axs[i, 2].imshow(img)
        axs[i, 2].imshow(mask, alpha=0.3)
        
        axs[i, 2].grid(True)
    plt.show()

show_image_randomly_chosen(df, n=3)

## Checking a (random) image

In [ ]:
def check_img(df, idx=None):
    if idx is None:
        idx = df.sample().index[0]
    
    img_ck = cv2.imread(df.img_path[idx])
    msk_ck = cv2.imread(df.mask_path[idx])
    img_ck = cv2.cvtColor(img_ck, cv2.COLOR_BGR2RGB)
    img_ck = img_ck / 255
    msk_ck = msk_ck / 255
    
    fig, axs = plt.subplots(1, 2, figsize=[12, 15])
    axs[0].imshow(img_ck)
    axs[0].set_title(f"maximum and minimum pixel values: {img_ck.max()}, {img_ck.min()}")

    axs[1].imshow(msk_ck, cmap="gray", alpha=.1)
    axs[1].set_title(f"maximum and minimum pixel values: {msk_ck.max()}, {msk_ck.min()}")
    plt.show()
check_img(df, idx=2)

## Define data generators for training data set, validation data set and test data set

### Split the data (path of images) into training data set, validation data set and test data set

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=2, shuffle=True)
df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=42, shuffle=True)
print("Shape of training data set: ", df_train.shape)
print("Shape of validation data set: ", df_val.shape)
print("Shape of test data set: ", df_test.shape)

### Image data generator and data augmentation

### Config

In [ ]:
batch_size = 32
img_height = 256
img_width = 256

augmentation_dict = dict(
    rotation_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=(0.2,1.3),
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

### Define data generator function

In [ ]:
def make_data_generator(
    dataframe,
    batch_size,
    augmentation_dict,
    target_size=(img_height, img_width),
    image_x_col="img_path",
    mask_x_col="mask_path",
    image_color_mode="rgb",
    mask_color_mode="grayscale",
    image_save_prefix="image",
    mask_save_prefix="mask",
    save_to_dir=None,
    seed=2
):
    
    # ImageDataGenerator generate batches of tensor image data with real-time data augmentation.
    _image_data_generator = ImageDataGenerator(**augmentation_dict)
    _mask_data_generator = ImageDataGenerator(**augmentation_dict)
    
    image_data_generator = _image_data_generator.flow_from_dataframe(
        dataframe=dataframe,
        x_col=image_x_col,
        target_size=target_size,
        color_mode=image_color_mode,
        class_mode=None,  # modes for yielding the target, None means no targets returned
        batch_size=batch_size,
        seed=seed,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix
    )
    mask_data_generator = _mask_data_generator.flow_from_dataframe(
        dataframe=dataframe,
        x_col=mask_x_col,
        target_size=target_size,
        color_mode=mask_color_mode,
        class_mode=None,
        batch_size=batch_size,
        seed=seed,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix
    )
    
    for (img, mask) in zip(image_data_generator, mask_data_generator):
        img, mask = normalise_and_set_threshold(img, mask)
        yield (img, mask)

def normalise_and_set_threshold(image, mask):
    image = image/255
    mask = mask/255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (image, mask)

### Get data generator

In [ ]:
train_data_generator = make_data_generator(
    df_train,
    batch_size,
    augmentation_dict,
    target_size=(img_height, img_width)
)
val_data_generator = make_data_generator(
    df_val,
    batch_size,
    dict(),
    target_size=(img_height, img_width)
)
test_data_generator = make_data_generator(
    dataframe=df_test,
    batch_size=batch_size,
    augmentation_dict=dict(),
    target_size=(img_height, img_width)
)

## Define Losses and metrics

In [ ]:
def iou(y_true, y_pred, smooth=K.epsilon()):
    """
    Intersection over Union (IoU) or Jaccard index (Jaccard similarity coefficient)
    for segmentation maps.
    
        TP / (TP + FP + FN)
    """
    intersection = K.sum(y_true * y_pred)  # TP
    union = K.sum(y_true) + K.sum(y_pred) - intersection  # (TP + FN) + (TP + FP) - TP
    return (intersection + smooth) / (union + smooth)
   
def iou_loss(y_true, y_pred, smooth=K.epsilon()):
    return 1 - iou(y_true, y_pred, smooth=smooth)

def dice_coef(y_true, y_pred, smooth=K.epsilon()):
    """
    Sørensen–Dice coefficient
    
        2 * TP / (2 * TP + FP + FN)
    """
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)  # TP
    y_true_area = K.sum(y_true)  # TP + FN
    y_pred_area = K.sum(y_pred)  # TP + FP
    combined_area = y_true_area + y_pred_area  # 2 * TP + FP + FN
    return (2 * intersection + smooth) / (combined_area + smooth)

def dice_loss(y_true, y_pred, smooth=K.epsilon()):
    return 1 - dice_coef(y_true, y_pred, smooth=smooth)

## Define model

### Unet

In [ ]:
def conv2d_block(
    tensor,
    num_filters,
    kernel_size=(3, 3),
    stack_num=2,
    batch_norm=True,
    activation_func="relu"
):
    """https://github.com/yingkaisha/keras-unet-collection/blob/d30f14a259656d2f26ea11ed978255d6a7d0ce37/keras_unet_collection/layer_utils.py#L197"""
    bias_flag = not batch_norm
    
    for i in range(stack_num):
        # linear convolution
        tensor = Conv2D(
            filters=num_filters,
            kernel_size=kernel_size,
            use_bias=bias_flag,
            kernel_initializer="he_normal",
            padding="same")(tensor)
        # batch normalization
        if batch_norm:
            tensor = BatchNormalization(axis=3)(tensor)
        # activation
        tensor = Activation(activation_func)(tensor)
    return tensor

def encoder_block(input_tensor, num_filters, dropout_rate=0.1):
    tensor4concat = conv2d_block(input_tensor, num_filters)
    tensor_pooling = MaxPooling2D(pool_size=(2, 2))(tensor4concat)
    tensor_pooling = Dropout(dropout_rate)(tensor_pooling)
    return tensor4concat, tensor_pooling

def decoder_block(input_tensor, filters, concat_layer, dropout_rate=0.1):
    x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding="same")(input_tensor)
    x = concatenate([x, concat_layer], axis=-1)
    x = Dropout(dropout_rate)(x)
    x = conv2d_block(x, filters)
    return x

def unet(img_shape=(img_height, img_width, 3), num_class=1):
    """
    Parameters
    ----------
    img_shape : tuple
        Shape of the image.
    num_class : int
        Output class number.
    
    Returns
    -------
      model : keras.engine.functional.Functional
          UNet model.
    """
    input_img = Input(shape=img_shape, name='main_input')
    conv1_1, pool1 = encoder_block(input_img, 64)
    conv2_1, pool2 = encoder_block(pool1, 128)
    conv3_1, pool3 = encoder_block(pool2, 256)
    conv4_1, pool4 = encoder_block(pool3, 512)
    conv5_1 = conv2d_block(pool4, 1024)
    conv4_2 = decoder_block(conv5_1, 512, conv4_1)
    conv3_3 = decoder_block(conv4_2, 256, conv3_1)
    conv2_4  = decoder_block(conv3_3, 128, conv2_1)
    conv1_5 = decoder_block(conv2_4, 64, conv1_1)
    unet_output = Conv2D(
        num_class, (1, 1),
        activation='sigmoid',
        padding='same',
        kernel_initializer='he_normal',
        kernel_regularizer=l2(3e-4),
        name='output'
    )(conv1_5)
    model = Model(inputs=[input_img, ], outputs=[unet_output, ], name="Unet")
    return model

### Define Optimizer and Callbacks

In [ ]:
steps_per_epoch = len(df_train) // batch_size
val_steps = len(df_val) / batch_size

clr = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate=1e-4,
    maximal_learning_rate=0.5,
    scale_fn=lambda x: 1,
    step_size=0.5 * steps_per_epoch
)
optimizer = tf.keras.optimizers.SGD(
    learning_rate=clr,
    momentum=0.01,
    nesterov=False,
    name='SGD',
)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('mri_seg_keras_unet.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

## Compile model and training

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    model = unet()
# model.summary()
model.compile(
    optimizer=optimizer,
    loss=dice_loss, 
    metrics=["binary_accuracy", iou, dice_coef],
)

In [ ]:
epochs = 150

history = model.fit(
    train_data_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    callbacks=callbacks, 
    validation_data=val_data_generator,
    validation_steps=val_steps,
)

## Post training visualisation

In [ ]:
history_post_training = history.history

train_dice_coeff_list = history_post_training['dice_coef']
val_dice_coeff_list = history_post_training['val_dice_coef']

train_jaccard_list = history_post_training['iou']
val_jaccard_list = history_post_training['val_iou']

train_loss_list = history_post_training['loss']
val_loss_list = history_post_training['val_loss']

plt.figure(figsize=(8, 6))
plt.figure(1)
plt.plot(train_loss_list, 'b-')
plt.plot(val_loss_list, 'r-')

plt.xlabel('iterations')
plt.ylabel('loss')
plt.title('Loss graph', fontsize=12)
plt.legend(['train', 'val']);

plt.figure(figsize=(8, 6))
plt.figure(2)
plt.plot(train_dice_coeff_list, 'b-')
plt.plot(val_dice_coeff_list, 'r-')

plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.title('Dice coefficient graph', fontsize=12)
plt.legend(['train', 'val']);
plt.show()

## Load trained model

In [ ]:
model.load_weights("mri_seg_keras_unet.h5")
# We can't load the model with the following method, because we only save the weights
# model = load_model(
#     "mri_seg_keras_unet.h5",
#     custom_objects={
#         "dice_loss": dice_loss,
#         "iou": iou,
#         "dice_coef": dice_coef
#     }
# )

results = model.evaluate(test_data_generator, steps=len(df_test) / batch_size)
print("Test loss: ", results[0])
print("Test IOU: ", results[2])
print("Test Dice Coefficent: ",results[3])

## Show prediction on test data set

In [ ]:
try:
    count = 0
    fig, axs = plt.subplots(10, 4, figsize = (20, 50), facecolor='w')
    for i in range(10):
      index = np.random.randint(0, len(df_test)) 
      image_path = df_test['img_path'].iloc[index]
      mask_path = df_test['mask_path'].iloc[index]
      image = cv2.imread(image_path)
      mask = cv2.imread(mask_path)
      img = cv2.resize(image, (img_height, img_width))
      img = img/255
      img = img[np.newaxis,:,:,:]
      pred=model.predict(img)

      axs[count][0].title.set_text('Original Image')
      axs[count][0].imshow(np.squeeze(img))

      axs[count][1].title.set_text('Original Mask')
      axs[count][1].imshow(mask, cmap = 'gray')

      axs[count][2].title.set_text('Prediction')
      axs[count][2].imshow(np.squeeze(pred))

      axs[count][3].title.set_text('Binary Prediction')
      axs[count][3].imshow(np.squeeze(pred) > 0.5)
      count+=1
    fig.tight_layout()
except Exception as e:
    print(str(e))

## References
- [Kaggle Notebook: brain_mri_segmentation](https://www.kaggle.com/code/irfanqasim/brain-mri-segmentation/notebook)
- [Kaggle Notebook: AttentionUNET_keras](https://www.kaggle.com/code/shahbaz92fast/attentionunet-keras)
- [Kaggle Notebook: Final Machine Learning](https://www.kaggle.com/code/luyen0/final-machine-learning)
- [Kaggle Notebook: Brain Tumor](https://www.kaggle.com/code/ziadhussien/brain-tumor)
- [影像切割任務常用的指標-IoU和Dice coefficient](https://chih-sheng-huang821.medium.com/%E5%BD%B1%E5%83%8F%E5%88%87%E5%89%B2%E4%BB%BB%E5%8B%99%E5%B8%B8%E7%94%A8%E7%9A%84%E6%8C%87%E6%A8%99-iou%E5%92%8Cdice-coefficient-3fcc1a89cd1c)
- [GitHub: yingkaisha/keras-unet-collection](https://github.com/yingkaisha/keras-unet-collection)
- [GitHub: JunMa11/SegLossJunMa11/SegLoss](https://github.com/JunMa11/SegLoss)
- [An overview of semantic image segmentation.](https://www.jeremyjordan.me/semantic-segmentation/)
- [3 Common Loss Functions for Image Segmentation](https://dev.to/_aadidev/3-common-loss-functions-for-image-segmentation-545o)